In [14]:
def ocs():
    
    import os
    import time
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver import DesiredCapabilities
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    from selenium.webdriver.common.keys import Keys
    import pandas as pd

    capabilities = DesiredCapabilities.CHROME.copy()
    capabilities['acceptSslCerts'] = True
    capabilities['acceptInsecureCerts'] = True

    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument('--window-size=1280,1696')
    #chrome_options.add_argument('--start-maximized')

    driver = webdriver.Chrome(executable_path=os.path.abspath('chromedriver'), options=chrome_options,
                              desired_capabilities=capabilities)

    driver.get('http://10.1.5.82/ocsreports/?function=visu_computers')
    login = driver.find_element_by_id('LOGIN').send_keys('admin')
    senha = driver.find_element_by_id('PASSWD')
    senha.send_keys('admin')
    senha.send_keys(Keys.ENTER)

    driver.find_element_by_xpath('/html/body/div/form[1]/div[2]/div/div/div[1]/div[1]/label/select/option[7]').click()
    time.sleep(2)
    table_id = driver.find_element(By.XPATH, '//*[@id="list_show_all"]/tbody')
    rows = table_id.find_elements(By.TAG_NAME, "tr")

    listaHyperlink = []
    listaAtivos = []

    for row in rows:

        div = row.find_element_by_tag_name("a")
        nome = row.find_element_by_tag_name("a").text
        hyperlink = div.get_attribute('href')
        listaHyperlink.append(hyperlink)

    list_ativo = []
    list_desc = []

    for link in listaHyperlink:
        driver.get(link)
        hardware = driver.find_element_by_xpath('/html/body/div/div[1]/ul/li[2]/a').click()
        storage = driver.find_element(By.XPATH, '//*[@id="affich_drives"]/tbody')
        rowst = storage.find_elements(By.TAG_NAME, "tr")

        for row in rowst:
            ativo = driver.find_element_by_xpath('/html/body/div/div[2]/h3')
            list_ativo.append(ativo.text)
            list_desc.append(row.text)
            
    df = pd.DataFrame({'nome': list_ativo, 'desc': list_desc})
    driver.quit()

    df['hd'] = df.desc.str[0:3]
    df = df[(df.nome.str.contains('SR|CF|CLF'))&(df.desc.str.contains('NTFS'))]
    df['desc'] = df.desc.str.replace('\w: ','').str.replace(' NTFS ', '|NTFS|').str.replace('\n\d\d?%', '').str.split('|')
    df['hd_desc'] = df.desc.str[0]
    df['free_space'] = df.desc.str[2].str.split(' ').str[0]
    df['total_space'] = df.desc.str[2].str.split(' ').str[1]
    df.drop(columns='desc', inplace=True)
    df.reset_index(drop=True, inplace=True)
    df['tipo'] = df.nome.apply(
        lambda x: 'Desktop' if x[0:1]=='D' else x).apply(
        lambda x: 'Notebook' if x[0:1]=='N' else x).apply(
        lambda x: 'Servidor' if x[0:1]=='C' else x).apply(
        lambda x: 'Servidor' if x[0:1]=='S' else x)    
    df[['free_space', 'total_space']] = df[['free_space', 'total_space']].apply(pd.to_numeric)
    df['used_space'] = df.total_space-df.free_space
    df = df[['tipo', 'nome', 'hd', 'hd_desc', 'used_space', 'free_space', 'total_space']]
    df['used_space'] = round(df.used_space/1024,4)
    df['free_space'] = round(df.free_space/1024,4)
    df['total_space'] = round(df.total_space/1024,4)
    df['%_used'] = round(df.used_space/df.total_space,4)
    df['%_free'] = round(df.free_space/df.total_space,4)
    df.sort_values(by=['tipo', 'nome'], inplace=True)
    df.to_csv('X:\TI\Documentos\Relatorio_Espaço_em_disco\Relatorio\R_Espaço_em_disco.csv', index=False)
    return df

In [15]:
start = time.perf_counter()
df = ocs()
end = time.perf_counter()
print(f'\n\nTempo de execução em: {str((end-start)//60)[:-2]} minutos',
      f'e {str(round((end-start)%60,0))[:-2]} segundos!\n\n')

ElementClickInterceptedException: Message: element click intercepted: Element <a href=" ?function=computer&amp;head=1&amp;systemid=2&amp;cat=hardware">...</a> is not clickable at point (150, 289). Other element would receive the click: <div style="text-align: center">...</div>
  (Session info: headless chrome=83.0.4103.116)


In [11]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [12]:
df.sort_values(by=['tipo', 'nome'], inplace=True)